In [203]:
import pandas as pd
import re
import nltk
import numpy as np
#nltk.download('all')
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download("omw-1.4")
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [158]:
with open ("../dat/parsed_books/mml-book.txt", encoding = "utf8") as f:
    text = f.readlines()
text = "".join(text)
print(len(text))
text = text.replace('©2021 M. P. Deisenroth, A. A. Faisal, C. S. Ong. Published by Cambridge University Press (2020).'," ")
text = text.replace("\n©by M. P. Deisenroth, A. A. Faisal, and C. S. Ong, 2021. https://mml-book.com ."," ")
text = text.replace('Draft (2022-01-11) of “Mathematics for Machine Learning”. Feedback: https://mml-book.com .'," ")
text = text.replace("This material is published by Cambridge University Press as Mathematics for Machine Learning by\nMarc Peter Deisenroth, A. Aldo Faisal, and Cheng Soon Ong (2020). This version is free to view\nand download for personal use only. Not for re-distribution, re-sale, or use in derivative works."," ")
print(len(text))


859754
817842


In [159]:
part1 = text.split("\nPart II\nCentral Machine Learning Problems")[0]
len(part1)

467726

In [160]:
chapters = ["\n1\nIntroduction and Motivation\n","\n2\nLinear Algebra\n","\n3\nAnalytic Geometry\n","\n4\nMatrix Decompositions\n",
            "\n5\nVector Calculus\n","\n6\nProbability and Distributions\n","\n7\nContinuous Optimization\n"]

chapters_dict ={}
rest_chapters = part1 ## .split(chapters[0])[1]

# Chapter 0 (not in index): intro
# chapter 1: ...
# ...
# Chapter n : ....
for idx,chapter_name in enumerate(chapters):
    text_splitted = rest_chapters.split(chapter_name)
    if idx ==0:
        pass
    else:
        chapters_dict[chapters[idx-1][3:-1]] = text_splitted[0]
    print("chapter: "+ chapter_name)
    rest_chapters = text_splitted[1]
chapters_dict[chapters[idx][3:-1]] = text_splitted[1]

chapter: 
1
Introduction and Motivation

chapter: 
2
Linear Algebra

chapter: 
3
Analytic Geometry

chapter: 
4
Matrix Decompositions

chapter: 
5
Vector Calculus

chapter: 
6
Probability and Distributions

chapter: 
7
Continuous Optimization



In [172]:
data = pd.DataFrame.from_dict(chapters_dict, orient='index',columns=["text"])
data

,text
Introduction and Motivation,Machine learning is about designing algorithms...
Linear Algebra,"When formalizing intuitive concepts, a common ..."
Analytic Geometry,"In Chapter 2, we studied vectors, vector space..."
Matrix Decompositions,"In Chapters 2 and 3, we studied ways to manipu..."
Vector Calculus,Many algorithms in machine learning optimize a...
Probability and Distributions,"Probability, loosely speaking, concerns the st..."
Continuous Optimization,Since machine learning algorithms are implemen...


In [197]:
def prep(text,delete_stopwords=True):
    #text = re.sub("\n"," ",text)
    #text = re.sub("\d+", "", text)

    # words only
    regex=u"[A-Za-z]+"
    regex = re.compile(regex)
    text = " ".join(regex.findall(text))
    # lower
    text = text.lower()
    lemmatizer = WordNetLemmatizer()
    word_tokens = nltk.word_tokenize(text)
    # remove stop words + lemmatization
    if delete_stopwords:
        stop_words = set(stopwords.words('english'))
        text = " ".join([lemmatizer.lemmatize(w) for w in word_tokens if not w in stop_words])
    else:
        text = " ".join([lemmatizer.lemmatize(w) for w in word_tokens])  
    return text

data["text_prep"] = data.text.apply(prep)

#### TF-IDF TRANSFORMATION

In [257]:
# !!! tf-idf 

tfidf = TfidfVectorizer(ngram_range=(1,3))
tfidf_matrix = tfidf.fit_transform(data.text_prep.values)
print(tfidf_matrix.shape)

vocab = {v:k for k,v in tfidf.vocabulary_.items()}

data["tfidf_topwords"]=None
for n,chapter in enumerate(data.index):
    s = ""
    for idx in np.argsort(tfidf_matrix.A[n,:])[::-1][:50]:
        s += ", "+vocab[idx]
    data.loc[chapter,"tfidf_topwords"] = s[2:]

    print(chapter+": \n ", data.loc[chapter,"tfidf_topwords"])

(7, 65421)
Introduction and Motivation: 
  learning, data, machine, machine learning, model, book, pillar, vector, concept, chapter, part, mathematical concept, foundation, mathematical, unseen, predictor, label, read, training, two, regression, way, learning system, machine learning system, read book, part ii, way read book, way read, pillar machine, two way read, four pillar machine, four pillar, pillar machine learning, data vector, unseen data, parameter, nd, well, input, density, system, algorithm, mean, estimation, two way, motivation, ing, classi cation, machine learning algorithm, goal
Linear Algebra: 
  vector, linear, matrix, column, basis, space, equation, mapping, subspace, system linear, pivot, system linear equation, linear equation, system, vector space, de, row, linearly, set, coordinate, solution, pivot column, linear mapping, ne, element, rn, linear algebra, transformation, respect, echelon form, echelon, algebra, form, row echelon form, row echelon, af, af ne, transf

#### SIMPLE BAG OF WORDS (COUNTVECTORIZER) + POS

In [194]:
from collections import Counter
Counter(data.text_prep['Linear Algebra'].split(" ")).most_common()[:15]

[('x', 404),
 ('vector', 299),
 ('b', 278),
 ('linear', 240),
 ('matrix', 181),
 ('c', 150),
 ('v', 135),
 ('space', 116),
 ('column', 115),
 ('equation', 112),
 ('r', 108),
 ('basis', 104),
 ('system', 103),
 ('mapping', 97),
 ('n', 90)]

In [262]:
# POS tagging 
data["text_prep_stopwords_incl"] = data.text.apply(prep,delete_stopwords=False)

def filter_by_POS(text):
    tokens = nltk.word_tokenize(text)
    pos = nltk.pos_tag(tokens)
    text = [p[0] for p in pos if p[1] in ["VBG","RB","RBR","RBS","JJ","JJR","JJS","NN","NNS"]]
    return " ".join(text)

data["text_nouns_adj_adv"] = data["text_prep_stopwords_incl"].apply(filter_by_POS)

In [265]:
data["text_nouns_adj_adv"]["Introduction and Motivation"]

'machine learning designing algorithm automatically valuable information data emphasis here automatic i machine learning general purpose methodology many datasets producing something mean ingful core machine learning data model learning machine learning inherently data driven data core data machine learning goal machine learning general purpose methodology valuable pattern data ideally much domain speci c expertise example large corpus document e g book many library machine learning method automatically nd relevant topic document hoffman al goal mod typically process data similar dataset example regression setting model function input real output model data formance task improves data account goal good model well yet unseen data future learning learning way automatically pattern structure data optimizing parameter model machine learning ha many success story software readily available rich exible machine learning system mathematical foundation machine learn ing important order fundamen

In [271]:
countvect = CountVectorizer(analyzer='word',ngram_range=(1, 2))
countvect_matrix = countvect.fit_transform(data.text_nouns_adj_adv.values)
print(countvect_matrix.shape)

vocab = {v:k for k,v in countvect.vocabulary_.items()}

data["countvect_topwords"]=None
for n,chapter in enumerate(data.index):
    s = ""
    for idx in np.argsort(countvect_matrix.A[n,:])[::-1][:50]:
        s += ", "+vocab[idx]
    data.loc[chapter,"countvect_topwords"] = s[2:]

    print(chapter+": \n ", data.loc[chapter,"countvect_topwords"])

(7, 25348)
Introduction and Motivation: 
  learning, data, machine, machine learning, model, book, vector, concept, chapter, part, mathematical, way, well, pillar, regression, system, not, matrix, reduction, parameter, cation, estimation, foundation, training, more, label, often, classi cation, ing, many, dimensionality, real, mathematical concept, predictor, density, component, dimensionality reduction, classi, algorithm, using, density estimation, input, function, numerical, goal, learning system, optimization, word, linear, however
Linear Algebra: 
  vector, linear, matrix, space, column, equation, basis, system, mapping, solution, vector space, linear equation, subspace, set, respect, row, system linear, transformation, form, coordinate, element, example, linearly, rn, pivot, following, linear mapping, not, inverse, then, af, algebra, rm, ne, remark, operation, multiplication, linear algebra, only, combination, group, pivot column, figure, base, independent, af ne, echelon, echelon